In [1]:
# signature = "{'dag_id': 'smc_daily_import', 'task_id': 'weblog_index_update', 'owner': 'icarus_so', 'nb_path': '/app/ws-ic/20200622_ssl_schema/update_domain_index.ipynb', 'execution_date': '2022-03-12'}"

#### 0. Initialization

In [2]:
from smtutil import hiveutil
import pandas as pd

Connect to Oracle Successfully
Connect to Oracle Successfully


In [3]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [4]:
sc, spark = hiveutil.get_sc_spark()

#### 1. Update Domain Index

##### 1.1. Calc domain - count(*) of the last date

In [5]:
spark.sql("MSCK REPAIR TABLE prd_biz_summ_vw.vw_idp_ssl_sed_call_summ")

DataFrame[]

In [6]:
spark.sql("REFRESH TABLE prd_biz_summ_vw.vw_idp_ssl_sed_call_summ")

DataFrame[]

In [7]:
max_date_id = spark.sql("""
select max(date_id) as max_date_id
from prd_biz_summ_vw.vw_idp_ssl_sed_call_summ
""").first()['max_date_id']

In [8]:
spark.sql("""
select
    ssl_cert_domain,
    count(*) as c
from prd_biz_summ_vw.vw_idp_ssl_sed_call_summ
where date_id={max_date_id}
group by ssl_cert_domain
having c >= 50
""".format(
    max_date_id=max_date_id
)).cache().createOrReplaceTempView("t_domain_count")

In [9]:
spark.sql("""
select
    ifnull(t_domain_count.ssl_cert_domain, t_domain_index.domain) as domain,
    t_domain_index.domain_index as o_domain_index,
    row_number() over (partition by 1 order by t_domain_index.domain_index nulls last) as domain_index
from t_domain_count
full outer join weblog.t_domain_index
    on t_domain_count.ssl_cert_domain = t_domain_index.domain
order by c desc
""").cache().createOrReplaceTempView("t_domain_index_new")

In [10]:
spark.table('t_domain_index_new').write.mode('overwrite').saveAsTable('weblog.t_domain_index_new')

#### 2. Update t_domain_dictionary

In [11]:
spark.sql("""
with t_domain_dictionary_new as (
    select
        '<Unclassified>' as tag_gp,
        '<Unclassified>' as tag,
        t_domain_index_new.domain as domain,
        'domain' as agg_level,
        domain_index as agg_value
    from t_domain_index_new
    left join weblog.t_domain_dictionary
        on t_domain_index_new.domain_index = t_domain_dictionary.agg_value
    where agg_value is null
)
select * from t_domain_dictionary_new union all
select * from weblog.t_domain_dictionary
""").write.mode('overwrite').saveAsTable('weblog.t_domain_dictionary_new')

#### 3. Rename New Tables

##### 3.1. t_domain_index_new

In [12]:
spark.sql("""drop table if exists weblog.t_domain_index_backup""")
spark.sql("""ALTER table weblog.t_domain_index RENAME TO weblog.t_domain_index_backup""")
spark.sql("""ALTER table weblog.t_domain_index_new RENAME TO weblog.t_domain_index""")

DataFrame[]

##### 3.2. t_domain_dictionary_new

In [13]:
spark.sql("""drop table if exists weblog.t_domain_dictionary_backup""")
spark.sql("""ALTER table weblog.t_domain_dictionary RENAME TO weblog.t_domain_dictionary_backup""")
spark.sql("""ALTER table weblog.t_domain_dictionary_new RENAME TO weblog.t_domain_dictionary""")

DataFrame[]

In [14]:
spark.stop()

In [15]:
sc.stop()